In [3]:
import numpy as np
import pandas as pd
import os
import random
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
register_matplotlib_converters()

from influxdb_client import InfluxDBClient

import pandas as pd

from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow import keras
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from numpy import array

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from statsmodels.tsa.vector_ar.var_model import VAR

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [6]:
#Auth parameters
my_token = "yaKfFeAsha8tNAZxvYeZBMmq-khO8tz-6Ut_PARgohiWzeW2j8BB86ND33Qbq7hR8bylmOROPQAUr-7M103_Yw=="
my_org = "iot-org"
bucket = "iot-demo"
client = InfluxDBClient(url="http://192.168.1.100:9999", token=my_token, org=my_org, debug=False)

In [9]:
days = '1d'
n_days = 1
window = '5m'

queryTemperature = '''
from(bucket: "iot-demo")
  |> range(start: -%s, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "temperature")
  |> filter(fn: (r) => r["device"] == "RoomWeather")
  |> aggregateWindow(every: %s, fn: median)
  ''' % (days, window)

queryHumidity = '''
from(bucket: "iot-demo")
  |> range(start: -%s, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "humidity")
  |> filter(fn: (r) => r["device"] == "RoomWeather")
  |> aggregateWindow(every: %s, fn: median)
  ''' % (days, window)

queryPressure = '''
from(bucket: "iot-demo")
  |> range(start: -%s, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "pressure")
  |> filter(fn: (r) => r["device"] == "RoomWeather")
  |> aggregateWindow(every: %s, fn: median)
  ''' % (days, window)

queryPeople = '''
from(bucket: "iot-demo")
  |> range(start: -%s, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "people")
  |> filter(fn: (r) => r["device"] == "peopleCounter")
  |> toFloat()
  |> aggregateWindow(every: %s, fn: median)
  ''' % (days, window)

In [13]:
dfTemp = client.query_api().query_data_frame(org=my_org, query=queryTemperature)

dfHum = client.query_api().query_data_frame(org=my_org, query=queryHumidity)

dfPress = client.query_api().query_data_frame(org=my_org, query=queryPressure)

#dfPeople = client.query_api().query_data_frame(org=my_org, query=queryPeople)

df = pd.DataFrame(dfTemp['_time']).rename(columns={'_time':'time'})
df['temperature'] = dfTemp['_value']
df['humidity'] = dfHum['_value']
df['pressure'] = dfPress['_value']
#df['people'] = dfPeople['_value']
df = df.set_index('time')
df = df.asfreq(freq='300S')

points_per_day = int(len(df)/n_days) 

df['temperature'] = df['temperature'].fillna(method='backfill').fillna(method='ffill')
df['humidity'] = df['humidity'].fillna(method='backfill').fillna(method='ffill')
df['pressure'] = df['pressure'].fillna(method='backfill').fillna(method='ffill')
#df['people'] = df['people'].fillna(method='backfill').fillna(method='ffill')

print(df['temperature'].isnull().sum())
print(df['humidity'].isnull().sum())
print(df['pressure'].isnull().sum())
#print(df['people'].isnull().sum())

0
0
0


In [11]:
multi_step_model = tf.keras.models.load_model('./24_step_model')
multi_step_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 96, 32)            4608      
_________________________________________________________________
lstm_3 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_1 (Dense)              (None, 24)                408       
Total params: 8,152
Trainable params: 8,152
Non-trainable params: 0
_________________________________________________________________


In [33]:
features_considered = ['pressure', 'temperature', 'humidity']
features = df[features_considered]
features.index = df.index
features.tail()

,pressure,temperature,humidity
time,,,
2020-07-17 15:35:00+00:00,1016.5,29.65,52.70
2020-07-17 15:40:00+00:00,1016.5,29.60,52.50
2020-07-17 15:45:00+00:00,1016.5,29.65,52.45
2020-07-17 15:50:00+00:00,1016.5,29.70,52.50
2020-07-17 15:55:00+00:00,1016.5,29.65,52.40


In [19]:
def multivariate_data(dataset, step):
    data = []

    start_index = 0
    end_index = len(dataset)
    
    indices = range(start_index, end_index, step)
    data.append(dataset[indices])
    
    return np.array(data)

In [55]:
dataset = features.values
#data_std = [2.76686183, 1.79688863, 4.73249127]
#data_mean = [1019.7463781 ,   29.91076259,   42.72058701]

data_mean = dataset.mean(axis=0)
data_std = dataset.std(axis=0)

dataset = (dataset-data_mean)/data_std

print(data_std)
print(data_mean)


[0.66733476 1.1757328  5.39177681]
[1016.19046586   29.33055556   50.75434028]


In [49]:
prediction_input = multivariate_data(dataset, 3)

In [50]:
to_be_predicted = tf.data.Dataset.from_tensor_slices(prediction_input).batch(1)

In [51]:
prediction = multi_step_model.predict(to_be_predicted)
prediction

array([[0.6262533 , 0.60630983, 0.65382725, 0.5979245 , 0.6408693 ,
        0.6085563 , 0.6750148 , 0.65362245, 0.6749866 , 0.7714676 ,
        0.71237904, 0.82908756, 0.7444074 , 0.74011254, 0.7548993 ,
        0.8071147 , 0.83650947, 0.8437196 , 0.8885894 , 0.8492024 ,
        0.7880054 , 0.9027425 , 0.8763484 , 0.8394311 ]], dtype=float32)

In [52]:
prediction_t = (data_std[1]*prediction) + data_mean[1]
prediction_t

array([[30.066862, 30.043413, 30.099281, 30.033554, 30.084045, 30.046055,
        30.124191, 30.099041, 30.124159, 30.237595, 30.168123, 30.30534 ,
        30.20578 , 30.20073 , 30.218115, 30.279507, 30.314066, 30.322544,
        30.3753  , 30.32899 , 30.257038, 30.39194 , 30.360907, 30.317501]],
      dtype=float32)

In [53]:
dti = pd.date_range('2020-07-17 15:55:00+00:00', periods=24, freq='300S')
dti

DatetimeIndex(['2020-07-17 15:55:00+00:00', '2020-07-17 16:00:00+00:00',
               '2020-07-17 16:05:00+00:00', '2020-07-17 16:10:00+00:00',
               '2020-07-17 16:15:00+00:00', '2020-07-17 16:20:00+00:00',
               '2020-07-17 16:25:00+00:00', '2020-07-17 16:30:00+00:00',
               '2020-07-17 16:35:00+00:00', '2020-07-17 16:40:00+00:00',
               '2020-07-17 16:45:00+00:00', '2020-07-17 16:50:00+00:00',
               '2020-07-17 16:55:00+00:00', '2020-07-17 17:00:00+00:00',
               '2020-07-17 17:05:00+00:00', '2020-07-17 17:10:00+00:00',
               '2020-07-17 17:15:00+00:00', '2020-07-17 17:20:00+00:00',
               '2020-07-17 17:25:00+00:00', '2020-07-17 17:30:00+00:00',
               '2020-07-17 17:35:00+00:00', '2020-07-17 17:40:00+00:00',
               '2020-07-17 17:45:00+00:00', '2020-07-17 17:50:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='300S')

In [54]:
rdf = pd.DataFrame(dti.values, columns = ['time'])
rdf['temperature'] = prediction_t[0]
rdf = rdf.set_index('time')
rdf


,temperature
time,
2020-07-17 15:55:00,30.066862
2020-07-17 16:00:00,30.043413
2020-07-17 16:05:00,30.099281
2020-07-17 16:10:00,30.033554
2020-07-17 16:15:00,30.084045
2020-07-17 16:20:00,30.046055
2020-07-17 16:25:00,30.124191
2020-07-17 16:30:00,30.099041
2020-07-17 16:35:00,30.124159
